# Datalab ITAM - Proyecto de Transparencia de Datos

**Objetivo:** extraer de datos de la [currícula y votaciones de la LXI legislatura](http://www.diputados.gob.mx/sistema_legislativo_LXI.html).
<br><br> Como herramienta fundamental hacemos uso de la librería [`scrapy`](https://docs.scrapy.org/en/latest/)

Este proceso se divide en las siguientes secciones:
1. [Definición del Spider](#Definición-del-Spider)
<br/><br/>
2. [Procesamiento de Datos](#Procesamiento-de-Datos)
    * [Limpieza Perfil General](#Limpieza-Perfil-General)
    * [Limpieza Proposiciones](#Limpieza-Proposiciones)
    * [Limpieza Iniciativas](#Limpieza-Iniciativas)
    * [Limpieza de la Ficha Curricular y las Comisiones](#Limpieza-de-la-Ficha-Curricular-y-las-Comisiones)
    * [Limpieza Votaciones](#Limpieza-Votaciones)
<br/><br/>
3. [Ejecución del Spider](#Ejecución-del-Spider)
<br/><br/>
4. [Creación de la base de datos](#Creación-de-la-base-de-datos)
<br/><br/>
5. [Visualizando los datos](#Visualizando-los-datos)

### Identificación de tablas
A continuación se presenta un caso ejemplo señalando cada una de las tablas a las que haremos referencia más adelante

**Perfil general:** La primera tabla por cada diputado corresponde a la siguiente sección de la currícula
<img src="Parte161.jpg" width="600">

**Iniciativas:** Podemos notar que, si bien tiene entradas con textos largos, no presenta una estructura muy compleja.
<img src="ParteIniciativas61.jpg" width="600">

**Proposiciones:** Notamos que es muy similar a la tabla de iniciativas, a diferencia de que contiene una columna extra
<img src="ParteProposiciones61.jpg" width="600">

**Ficha Curricular:** Para cada una de los apartados dentro de la ficha (Escolaridad, Administación Pública, Trayectoria Política, ...) se genera una lista de diccionarios por separado
<img src="ParteEscolaridad61.jpg" width="600">

Tomando el [caso ejemplo](http://sitllxi.diputados.gob.mx/curricula.php?dipt=260), la información se guardará en el diccionario ***diccionario_final*** y abreviando las secciones de proposicones e iniciativas, la estructura dentro del archivo quedaría de la siguiente forma:
```css
{
    "cristabellzamoracabrera": {
        "PERFIL GENERAL": [
            {
                'ClaveUnica': 'cristabellzamoracabrera',
                'Nombre': 'Cristabell Zamora Cabrera',
                'TipoElección': 'Mayoría Relativa',
                'Entidad': 'Tamaulipas',
                'Partido': 'PRI',
                'Distrito': '1',
                'Curul': 'I-302',
                'Correo': 'cristabell.zamora@congreso.gob.mx ',
                'FechaNacimiento': '16-noviembre-1979',
                'Suplente': 'Adolfo Víctor García Jiménez',
                'UltimoGrado': 'Mayoría Relativa ',
                'PaginaInternet': 'http://sitllxi.diputados.gob.mx/curricula.php?dipt=260'
            }
        ],
        "COMISIONES": [
            {
                'Tipo': 'ORDINARIA',
                'Comisión': ' Fortalecimiento al Federalismo  '
            },...
        ],
        "INICIATIVAS": [
            {
                'NombreIniciativa': 'Proyecto de decreto que reforma el artículo 2 de la Ley del Impuesto al Valor Agregado.',
                'TipoIniciativa': 'Suscribe',
                'FechaIniciativa': ' 23-Septiembre-2009',
                'TurnoComisionI': 'NA',
                'SinopsisIniciativa': 'Considerar como región fronteriza, al municipio de Valle Hermoso, Tamaulipas.',
                'TramiteIniciativa': 'Desechada (art. 89) 16-Julio-2012 ',
                'GacetaIniciativa': '23-Septiembre-2009'
            },...
        ],
        "PROPOSICIONES": [
            {
                'NombreProposicion': 'Punto de acuerdo por el que se solicita a las ...',
                'TipoProposicion': 'Suscribe',
                'FechaProposicion': ' 23-Septiembre-2009 ',
                'TurnoComisionP': 'NA',
                'ResolutivosProponente': 'PRIMERO.- Se solicita a las instituciones ... ",
                'ResolutivosAprobados': 'Primero. Se desecha la proposición con punto de ...',
                'TramiteProposicion': 'Desechada   30-Septiembre-2010 ',
                'GacetaProposicion': '23-Septiembre-2009'
            },...
        ],
        "ESCOLARIDAD": [
            {
                'ESCOLARIDAD': 'Maestría - Master of Business Administration',
                'Fechas': '2005-2007'
            },...
        ],
        "TRAYECTORIA POLÍTICA": [
            {
                'TRAYECTORIA POLÍTICA': 'Integrante de Asambleas Nacionales - PRI',
                'Fechas': 'NA'
            },...
        ],
        'CARGOS DE ELECCIÓN POPULAR': [
            {
                'CARGOS DE ELECCIÓN POPULAR': 'Regidor, Tamaulipas - Nuevo Laredo - PRI',
                'Fechas': '2002-2004'
            },...
        ],
        "ADMINISTRACIÓN PÚBLICA FEDERAL": [
            {
                'ADMINISTRACIÓN PÚBLICA FEDERAL': 'Directora de Atención a la Juventud - ...',
                'Fechas': '1996-1997'
            },...
        ],
        'ADMINISTRACIÓN PÚBLICA LOCAL': [
            {
                'ADMINISTRACIÓN PÚBLICA LOCAL': 'Coordinadora del Programa Manos Juveniles - Ayuntamiento de Nuevo Laredo',
                'Fechas': '1998'
            },...
        ],
        'ASOCIACIONES A LAS QUE PERTENECE': [
            {
                'ASOCIACIONES A LAS QUE PERTENECE': 'Miembro - Fundación Colosio A.C., filial Nuevo Laredo',
                'Fechas': '11/11/2005'
            }
        ],
        "ACTIVIDADES DOCENTES": [
            {
               'ACTIVIDADES DOCENTES': 'Invitado - Foro: ¿Por qué votar?',
                'Fechas': '10/06/2009'
            },...
        ],
        'PUBLICACIONES': [
            {
                'PUBLICACIONES': 'Política vs. Valores - Revista', 
                'Fechas': '01/11/2000'
            },...
        ],
}

```

**Votaciones:** La estructura de las páginas que contienen las votaciones de cada diputado se muestra a continuación
<img src="Votaciones61.jpg" width="600">

Toda la información de votaciones se guardará en un diccionario llamado ***por_votaciones***, el cual tendrá esencialmente la siguiente estructura:

```css
{'ppararcmcmcemmsacvsgmscur': {
        'ClaveUnicaVotacion': 'ppararcmcmcemmsacvsgmscur',
          'NombreVotacion': 'PROPOSICIÓN CON PUNTO DE ACUERDO, RELATIVO AL RETIRO DE LA CONCESIÓN DE LA MINA DE COBRE MEXICANA DE CANANEA A LA EMPRESA MINERA MÉXICO, S. A. DE C. V., SUBSIDIARIA DEL GRUPO MÉXICO. (SI SE CONSIDERA DE URGENTE RESOLUCIÓN) ',
          'FechaVotacion': '23 Septiembre 2009',
          'javiergilortiz': 'En contra',
          'julianfranciscovelazquezyllorente': 'Ausente',
          'cristabellzamoracabrera': 'En contra',
          'edgardomelhemsalinas': 'En contra',
          'rodolfotorrecantu': 'En contra',
          'baltazarmanuelhinojosaochoa': 'En contra',
          'jesuseverardovillarrealsalinas': 'En contra',
          'josefranciscorabagocastillo': 'En contra',
          'luisalejandroguevaracobos': 'En contra',
          'oralialopezhernandez': 'En contra',
          ...
    }
}


```

La key *'ppararcmcmcemmsacvsgmscur'* corresponde al identificador único de la votación. El value que corresponde a dicha key es un diccionario que contiene el nombre de la votación, su respectiva fecha y los sentidos de voto de cada uno de los diputados con respecto a dicha votación.

**Importaciones necesarias:**
- ***scrapy***
- ***re***: será indispensable para la limpieza de textos
- ***json***: la utilizaremos para la creación del archivo que contendrá la base de datos

In [1]:
import scrapy
import numpy as np
from scrapy.crawler import CrawlerProcess
import re
import json

### Definición del Spider

In [2]:
class Diputados(scrapy.Spider):
    '''
        En esta sección es donde se va hacer, principalmente, el trabajo de recolección de datos,
        para su posterior procesamiento.
    '''
    name = "spider_diputados"
    
    def start_requests(self):
        '''
            Hacemos el llamado a cada una de las páginas que vamos a scrappear
        '''
        links = [urlCurr + i for i in terminaciones]
        for link in links:
            yield scrapy.Request( url = link, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}, meta={'link':link},
                              callback = self.parse )
    
    def parse(self,response):
        '''
            En esta sección primero obtenemos la información del Perfil General y la vamos guardando en un diccionario
        '''
        link = response.meta['link']
        
        # Información del Perfil General
        nombre = response.css('span.Estilo67::text').extract_first()
        nombre = limpiaText(nombre)
        cumple = response.css('td.textoNegro::text').extract()[5]
        curp = creaCurp(nombre)    # Generamos el identifiador único
        dipus[curp]={}      # Declaramos el diccionario que contiene el Perfil General
        dipus.get(curp,{})['ClaveUnica']=curp
        dipus.get(curp,{})['Nombre']=nombre 
        dipus.get(curp,{})['TipoElección']=limpiaEsp(response.css('td.textoNegro::text').extract()[0])
        dipus.get(curp,{})['Entidad']=limpiaEsp(response.css('td.textoNegro::text').extract()[1])
        parti=response.xpath('//tr/td/table//td//img/@src').extract()[1]
        dipus.get(curp,{})['Partido']=obtienePartido(parti)
        dipus.get(curp,{})['Distrito']=limpiaEsp(response.css('td.textoNegro::text').extract()[2])
        dipus.get(curp,{})['Curul'] =limpiaEsp(response.css('td.textoNegro::text').extract()[4])
        dipus.get(curp,{})['Correo']=response.css('a.linkNegroSin::text').extract()[0]
        dipus.get(curp,{})['FechaNacimiento']=limpiaEsp(cumple)
        supl=response.css('span.Estilo67::text').extract()[6]
        dipus.get(curp,{})['Suplente']=limpiaText(supl)
        dipus.get(curp,{})['UltimoGrado']=response.xpath('//td[@class="textoNegro"]/text()').extract_first()
        dipus.get(curp,{})['PaginaInternet'] = link
        
        # Obtenemos la tabla de la ficha curricular sin procesar
        exp_lista=response.css("table table td").extract()
        exp_lista=exp_lista[26:len(exp_lista)]
        exp_texto = response.css("table table td::text").extract()
        exp_texto=exp_texto[11:len(exp_texto)]
        
        
        # Obtenemos la tabla de las comisiones sin procesar
        comisiones = response.xpath("//td[contains(@align, 'left')]").extract()[5]
        
        # Guardamos las 3 listas
        dic_experiencia_lista[curp] = exp_lista
        dic_experiencia_texto[curp] = exp_texto
        dic_comisiones[curp] = comisiones
        
        # Obtenemos el link que nos redirige a la página que contiene los periodos ordinarios y extraordinarios
        link_iniciativas=urlDipu + response.css('a.linkBlancoSin::attr(href)').extract()[0]
        link_proposiciones=urlDipu + response.css('a.linkBlancoSin::attr(href)').extract()[1]
        link_votaciones=urlDipu + response.css('a.linkBlancoSin::attr(href)').extract()[3]
        
        # Inicializamos los diccionarios donde vamos a guardar las proposiciones e iniciativas
        dic_proposiciones[curp] = []
        dic_iniciativas[curp]=[]
        
        # Mandamos el link de iniciativas recién obtenido 
        yield response.follow(url=link_iniciativas, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse6)
        # Mandamos el link de proposiciones recién obtenido 
        yield response.follow(url=link_proposiciones, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse8)
        # Mandamos el link de votaciones recién obtenido 
        yield response.follow(url=link_votaciones, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse4)
    
    def parse6(self,response):
        '''
            Dentro de esta sección se encuentran los links para cada uno de los periodos. Los guardamos y, uno por uno,
            los visitamos para obtener las iniciativas de cada periodo.
        '''
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse7)
    
    def parse7(self,response):    
        '''
            En esta sección extraemos la tabla de iniciativas del periodo en el que nos encontremos
        '''
        curp = response.meta['curp']
        listaInic = response.css('td.Estilo69').getall() # Obtenemos todas las entradas de la tabla
        dic_iniciativas[curp] = dic_iniciativas[curp]+ listaInic # Guardamos las entradas con el resto de los periodos
    
    def parse8(self,response):
        '''
            Dentro de esta sección se encuentran los links para cada uno de los periodos. Los guardamos y, uno por uno,
            los visitamos para obtener las proposiciones de cada periodo.
        '''
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse9)
    def parse9(self,response):  
        '''
            En esta sección extraemos la tabla de proposiciones del periodo en el que nos encontremos
        '''
        curp = response.meta['curp']
        listaPropos = response.css('td.Estilo69').getall() # Obtenemos todas las entradas de la tabla
        dic_proposiciones[curp]= dic_proposiciones[curp]+listaPropos # Guardamos las entradas con el resto de los periodos

        
    def parse4(self,response):
        '''
            Dentro de esta sección se encuentran los links para cada uno de los periodos. Los guardamos y, uno por uno,
            los visitamos para obtener las votaciones de cada periodo.
        '''
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse5)
                
    def parse5(self,response):
        '''
            En esta sección obtendremos los sentidos del voto del periodo en cuestión.
            El diccionario por_votaciones es el encargado de guardar en sus keys el identificador único de cada votación y 
            en sus values el diccionario que contiene el nombre de la votación, la fecha de la votación y el sentido del voto
            de cada uno de los diputados con respecto a dicha votación.
            La estructura de este diccionario por_votaciones es precisamente la mencionada en el ejemplo del principio del
            documento.
        '''
        curp = response.meta['curp']
        aux = response.xpath('//td[@class="smallVerde"]/text()').extract()
        todas_celdas=response.xpath('//td/text()').extract()[7:]
        j=2; 
        fechaVotacion=todas_celdas[0] 
        for i in range(1,len(aux),4):
            celda = todas_celdas[j] 
            votacion = response.xpath('//td[@class="smallVerde"]/text()').extract()[i] # Extraemos el nombre de la votación
            posi = response.xpath('//td[@class="smallVerde"]/text()').extract()[i+2]   # Extraemos el sentido del voto del diputado
            id_voto = limpiaVotacion(votacion)        # Obtenemos el identificador único de la votación
            if(id_voto in por_votaciones.keys()): 
                por_votaciones[id_voto][curp]=posi   # Si la votación ya se encontraba registrada, únicamente guardamos el sentido del voto
            else:       # Si la votación aún no se había registrado, guardamos tanto su nombre como su identificador único, así como el sentido del voto
                por_votaciones[id_voto] = {"ClaveUnicaVotacion":id_voto,"NombreVotacion":votacion, "FechaVotacion": fechaVotacion, curp:posi}
            if(i+4<len(aux)):       # Si encontramos una instancia de class = 'smallVerde', significa que las siguientes votaciones pertenecen a una nueva fecha
                if(todas_celdas[j+4]!=response.xpath('//td[@class="smallVerde"]/text()').extract()[i+4]): 
                    fechaVotacion=todas_celdas[j+3]  # cambiamos a la nueva fecha
                    j=j+5 
                else: 
                    j=j+4

## Procesamiento de Datos
### Limpieza Perfil General

In [3]:
def creaCurp(nombre):
    '''
        Como identificador único de cada diputado tomamos su nombre en minúsculas, sin espacios, acentos o símbolos
    '''
    curp = nombre.replace('.', '')
    curp = nombre.replace('-', '')
    curp = curp.replace(' ', '')
    curp = curp.replace('(DECESO)', '')
    curp = curp.replace('(Deceso)', '')
    curp = curp.lower()
    curp = curp.replace('á', 'a')
    curp = curp.replace('é', 'e')
    curp = curp.replace('í', 'i')
    curp = curp.replace('ó', 'o')
    curp = curp.replace('ú', 'u')
    #curp = curp.replace('ñ', 'n')
    return curp

In [4]:
def limpiaText(nom):
    '''
        Esta función nos sirve para limpiar las información obtenida, quitando ciertas cadenas específicas
    '''
    resp = re.sub('\s+',' ',nom)
    resp = resp.replace('Dip. ', '')
    resp = resp.replace('(LICENCIA)', '')
    resp = resp.replace('Suplente: ', '')
    resp = resp.replace('  ', ' ')
    if(resp[-1]==" "):
        resp = resp[0:len(resp)-1]    # Evita que nuestro string contenga un espacio al final
    if(resp[0]==" "):
        resp = resp[1:]               # Evita que nuestro string contenga un espacio al principio
    return resp

In [5]:
def limpiaEsp(nom):
    '''
        Similar a limpiaText, pero se enfoca únicamente en eliminar espacios innecesarios
    '''
    resp = re.sub('\s+',' ',nom)
    resp = resp.replace('  ', ' ')
    if(resp!=" "):
        if(resp[-1]==" "):
            resp = resp[0:len(resp)-1]
        if(resp[0]==" "):
            resp = resp[1:]
    return resp

In [6]:
def obtienePartido(imagen):
    '''
        Dado que el sitio del que obtenemos los datos (sitl.diputados.gob.mx) no indica
        el partido al que pertenece un diputado más que a través de una imagen, nos corresponde obtener dicho dato
        por medio del nombre de la imagen correspondiente.
        Esta función recibe el nombre de la imagen y devuelve en un string el partido del diputado.
    '''
    res=""
    if(imagen=='images/pan.png'):
        res = "PAN"
    elif(imagen=='images/logpt.jpg'):
        res = "PT"
    elif(imagen=='images/LogoMorena.jpg'):
        res = "Morena"
    elif(imagen=='images/encuentro.png'):
        res = "Encuentro Social"
    elif(imagen=='images/logo_movimiento_ciudadano.png'):
        res = "Movimiento Ciudadano"
    elif(imagen=='images/pri01.png'):
        res = "PRI"
    elif(imagen=='images/logvrd.jpg'):
        res = "Partido Verde"
    elif(imagen=='images/prd01.png'):
        res = "PRD"
    elif(imagen=='images/logo_SP.jpg'):
        res = "Sin Partido"
    elif(imagen=="images/panal.gif"):
        res = "Nueva Alianza"
    elif(imagen=="images/independiente.png"):
        res = "Independiente"
    elif(imagen=="images/pasc.jpg"):
        res = "PSD"
    elif(imagen=="images/convergencia.gif"):
        res = "Convergencia"
    else:
         res = "Sin Partido"
    return res
                                    

### Limpieza Proposiciones

In [7]:
def limpiaPropos(listaProp):
    '''
        Esta función recibe la tabla de proposiciones obtenida por nuestro
        Spider y se encarga de extaer la información de cada una de las 5 columnas.
        Cada renglón de la corresponde a una proposición y de cada columna se extraen múltiples datos.
        La 1a columna nos indica el nombre y tipo de la iniciativa (De grupo, proponente o Diversos Grupos Parlamentarios)
        La 2a columna nos dan las comisiones en turno y la fecha de presentación.
        La 3a columna nos da los resolutivos del proponente
        La 4a columna los resolutivos aprobados
        La 5a y última nos da el trámite en el que se encuentra la proposición y la fecha del mismo, así
        como la fecha de publicación en Gaceta
        Estos datos de cada proposición se guardan como diccionario, por lo que esta función devuelve
        la tabla de proposiciones condensada en una lista de diccionarios.
        Debido a la estructura tan diferente de cada una de las columnas, cada una requiere de su propio
        proceso de limpieza de texto
    '''
    n=len(listaProp)
    listaFinal=list()
    for i in range(0,n,5):
        renglon = listaProp[i:i+5]  # Seleccionamos las 5 columnas del renglón
        
        #Columna1
        casilla = renglon[0]        # La primera entrada del renglón corresponde a la primera columna
        resp = re.sub('\s+',' ',casilla)
        resp = resp.replace('<br>', '')
        resp = resp.replace('</span>', '')
        entrada=re.split('</b> |<span class="Estilo71">',resp)
        iniciativa=entrada[2]      # Extraemos el nombre de la proposición
        tipo = re.split('<b>',entrada[3])[0].replace(': ','')  # Obtenemos el tipo de proposición
                
        #Columna2     
        texto2=renglon[1]
        listaProp2=texto2.split('<span class="Estilo71">')
        casilla2 = listaProp2[1]
        resp2 = re.sub('\s+',' ',casilla2)
        resp2 = resp2.replace('<br>', '')
        resp2 = resp2.replace('</span>', '')
        resp2 = resp2.replace('</b>', '')
        resp2 = resp2.replace('</td>', '')
        entrada2=re.split('<b>',resp2)
        fechaPres = entrada2[1]         # Obtenemos la fecha de presentación
        if(2 in range(-len(entrada2), len(entrada2))):
            clasif=entrada2[2].replace('- ', '')     # Obtenemos la comisión en turno
        else:
            clasif="NA"      # No se especificó comisión en turno
        
        #Columna3
        casilla3=renglon[2]
        sinopsis = casilla3.replace('</td>', '')
        sinopsis = sinopsis.replace('</span>', '')
        sinopsis = sinopsis.replace('<br>', '')
        sinopsis=sinopsis.split('<span class="Estilo71">')[1]  # Obtenemos los resolutivos del proponente
        
        #Columna4
        casilla4=renglon[3]
        sinopsis2 = casilla4.replace('</td>', '')
        sinopsis2 = sinopsis2.replace('</span>', '')
        sinopsis2 = sinopsis2.replace('<br>', '')
        sinopsis2=sinopsis2.split('<span class="Estilo71">')[1]  # Obtenemos los resolutivos aprovados
        
        #Columna5
        casilla5=renglon[4]
        listaProp3=re.split('<span class="Estilo71">',casilla5)  
        resp3 = re.sub('\s+',' ',listaProp3[1])
        resp3 = resp3.replace('</td>', '')
        resp3 = resp3.replace('</span>', '')
        resp3 = resp3.replace(' </b>', '')
        resp3 = resp3.replace('<br>', '')
        status = resp3.replace('</a>', '')    
        ultimo = re.split('</b>',status)[0].replace('<b>', '')

        fecha_op = re.split('</b>',status)[1]
        fecha_op = re.sub('\s+',' ',fecha_op)
        fecha_op = fecha_op.replace('</td>', '')
        fecha_op = fecha_op.replace('</span>', '')
        fecha_op = fecha_op.replace('<b>', '')
        fecha_op = fecha_op.replace('<br>', '')
        fecha_op = fecha_op.replace('</a>', '')
        fecha_op = fecha_op.replace('con fecha', '')

        tramite = ultimo + fecha_op # Obtenemos el trámite en el que se encuentra la proposición con su respectiva fecha

        publicacion = listaProp3[2]  
        pub_final=re.split('<b>',publicacion)[1]
        pub_final = pub_final.replace('</td>', '')
        pub_final = pub_final.replace('</span>', '')
        pub_final = pub_final.replace('</b>', '')
        pub_final = pub_final.replace('<br>', '')
        pub_final = pub_final.replace('</a>', '')  # Extraemos la fecha de publicación

        listaFinal.append({"NombreProposicion": iniciativa,"TipoProposicion": tipo,"FechaProposicion":fechaPres,"TurnoComisionP":clasif,"ResolutivosProponente":sinopsis,"ResolutivosAprobados":sinopsis2,"TramiteProposicion":tramite, "GacetaProposicion":pub_final})
    return listaFinal

### Limpieza Iniciativas

In [8]:
def limpiaInici(listaProp):
    '''
        Esta función recibe la tabla de iniciativas obtenida por nuestro
        Spider y se encarga de extaer la información de cada una de las 4 columnas.
        Cada renglón de la corresponde a una iniciativa y de cada columna se extraen múltiples datos.
        La 1a columna nos indica el nombre y tipo de la iniciativa (De grupo, proponente o Diversos Grupos Parlamentarios)
        La 2a columna nos dan las comisiones en turno y la fecha de presentación.
        La 3a columna nos da la sinopsis
        La 4a y última nos da el trámite en el pleno y la fecha del mismo, así como la fecha de publicación en Gaceta
        Estos datos de cada iniciativa se guardan como diccionario, por lo que esta función devuelve la 
        tabla de iniciativas condensada en una lista de diccionarios.
        Debido a la estructura tan diferente de cada una de las columnas, cada una requiere de su propio
        proceso de limpieza de texto
    '''
    n=len(listaProp)
    listaFinal=list()
    for i in range(0,n,4):
        renglon = listaProp[i:i+4] # Seleccionamos las 4 columnas del renglón
        #Columna1
        casilla = renglon[0]     # La primera entrada del renglón corresponde a la primera columna
        resp = re.sub('\s+',' ',casilla)
        resp = resp.replace('<br>', '')
        resp = resp.replace('</span>', '')
        entrada=re.split('</b> |<span class="Estilo71">',resp)
        iniciativa=entrada[2]        # Extraemos el nombre de la proposición
        tipo = re.split('<b>',entrada[3])[0].replace(': ','')   # Obtenemos el tipo de proposición
        
        #Columna2
        texto2=renglon[1]
        listaProp2=texto2.split('</span>')
        casilla2 = listaProp2[0]#
        resp2 = re.sub('\s+',' ',casilla2)
        resp2 = resp2.replace('<br>', '')
        resp2 = resp2.replace('<span class="Estilo71">', '')
        resp2 = resp2.replace('</b>', '')
        resp2 = resp2.replace('</td>', '')
        entrada2=re.split('<b>',resp2)
        fechaPres = entrada2[1]     # Obtenemos la fecha de presentación
        
        
        
        resp3 = re.sub('\s+',' ',listaProp2[1])
        resp3 = resp3.replace('<br>', '')
        resp3 = resp3.replace('<span class="Estilo71">', '')
        resp3 = resp3.replace('</b>', '')
        resp3 = resp3.replace('</td>', '')
        entrada3=resp3.replace('<b>','')
        if(entrada3!=' '):
            if(entrada3[0]==' ' and entrada3[1]=='-' and entrada3[2]==' '):
                entrada3=entrada3[3:]
        else:
            entrada3='NA'
        clasif=entrada3      # Obtenemos la comisión en turno  
        
        
        #Columna3
        casilla3=renglon[2]
        sinopsis = casilla3.replace('</td>', '')
        sinopsis = sinopsis.replace('</span>', '')
        sinopsis=sinopsis.split('<span class="Estilo71">')[1]  # Obtenemos la sinopsis
        
        #Columna4
        casilla4=renglon[3]
        listaProp3=re.split('<b>',casilla4)
        resp3 = re.sub('\s+',' ',listaProp3[2])
        resp3 = resp3.replace('</td>', '')
        resp3 = resp3.replace('</span>', '')
        resp3 = resp3.replace('</b>', '')
        publicacion = resp3.replace('</a>', '')   # Obtenemos la fecha de publicación
        
        parte1 = re.split('</span>',listaProp3[1])
        parte1 = re.sub('</b>',' ', parte1[0])
        status = re.sub('<br>  con fecha','',parte1)
        status = re.sub('\s+',' ',status)
        
        
        if 'linkTitulo">' in status:   # Por si la fecha viene con link
            status = re.sub('</a>','',status)
            status = re.sub(' [<>].*[<>]','',status)
        
        tramite = status    # Obtenemos el trámite en el pleno con su respectiva fecha
        
        listaFinal.append({"NombreIniciativa": iniciativa,"TipoIniciativa": tipo,"FechaIniciativa":fechaPres,"TurnoComisionI":clasif,"SinopsisIniciativa":sinopsis,"TramiteIniciativa":tramite, "GacetaIniciativa":publicacion})
    return listaFinal

### Limpieza de la Ficha Curricular y las Comisiones

In [9]:
def obtieneComi(li):
    '''
        El listado de comisiones a las que pertenece un diputado proviene de una tabla independiente
        de aquella que contiene el perfil básico y aquella que contiene la ficha curricular. Esta función 
        recibe como único atributo la tabla de comisiones y se encarga de extraer el nombre de la comisión 
        y el tipo de la misma (Ordinaria, Grupo de Amistad, ...).
        Cada comisión se guarda en un diccionario, por lo que la función tiene como salida una lista de
        diccionarios.
    '''
    apartado=""
    comi=list()
    i=0
    while(i<len(li)-1):
        if 'span class="Estilo67">' in li[i]:    # Este 'Estilo67' nos indica cuando se encuentra el tipo de comisión
            s=li[i].split('"Estilo67">')
            apartado=s[1]               # Obtenemos el tipo
            i=i+1
        elif 'class="linkNegroSin">' in li[i]:   # Este 'linkNegroSin' nos indica cuando se encuentra el nombre de la comisión
            t=li[i].split('"linkNegroSin">')
            texto = t[1]                # Obtenemos el nombre
            comi.append({"Tipo":apartado,"Comisión":texto})
            i=i+1
        else:
            i=i+1
    return comi

In [10]:
def empataLis(li1,li2):
    '''
        Debido a la forma en la que la tabla de ficha curricular está estructurada, los datos faltantes crean
        un problema al querer organizar la información de la tabla. Esta función tiene como único objetivo rellenar
        los espacios en los que faltan datos con la notación 'NA', para que la estructura de la tabla se mantenga consistente
        y sea manejable en las siguientes partes del proceso'
    '''
    if(len(li1)!=len(li2)):
        for i in range(0,len(li1)):
            if '"></td>' in li1[i]: # Cuando hay un dato faltante, el código HTML no contiene texto y hace un salto de celda
                li2.insert(i, "NA") # por lo que rellenamos con la notación NA para evitar dicho salto de celda.
                if(len(li1)==len(li2)):
                    break
    return li2

In [11]:
def obtieneExp(curp,li1,li2,tabla_comi, liInic,liPropos,perfil_general):
    '''
        Esta función es la encargada de juntar toda la información de un diputado y guardarla en un diccionario.
        
        liInic y liPropos son las tablas de iniciativas y proposiciones, respectivamente. Se limpiarán y guardarán en listas
        de diccionarios por medio de las funciones limpiaInici y limpiaPropos antes declaradas.
        
        tabla_comi es la lista de comisiones sin procesar. Por medio de la función obtieneComi, se adquiere la lista de
        diccionarios con cada una de las comisiones
        
        perfil_general es el diccionario con la información básica ya obtenida. Este diccionario ya no requiere de mayor
        procesamiento.
        
        Los argumentos de entrada li1 y li2 proporcionarán la información de la ficha curricular con la ayuda de la
        función empataLis vista anteriormente.
        Para cada apartado de la ficha curricular (Escolaridad, Experiencia Legislativa, Trayectoria Política, ...) se genera
        una lista de diccionarios, donde cada diccionario es una entrada de cada apartado.
        
        Esta función devuelve un diccionario por diputado cuyas keys son los nombres de las tablas a guardar,
        los valores son sus respectivas listas de diccionarios. La estructura del diccionario de salida es precisamente la
        mencionada en el ejemplo al principio del documento.
    '''
    exp={"CURP":curp}
    exp={"PERFIL GENERAL":[perfil_general]}  # Empezamos guardando el perfil general
    tabla_comi_sep = tabla_comi.split("</")
    list_comis=obtieneComi(tabla_comi_sep)   # Procesamos la tabla de comisiones
    if list_comis!=[]:
        exp["COMISIONES"]=list_comis    # Guardamos las comisiones, si existen
    inics = limpiaInici(liInic)     # Procesamos la tabla de iniciativas
    props = limpiaPropos(liPropos)  # Procesamos la tabla de proposiciones
    exp["INICIATIVAS"]=inics        # Guardamos las iniciativas
    exp["PROPOSICIONES"]=props      # Guardamos las proposiciones
    
    if(len(list_comis)!=0):
        x = set()
        for i in list_comis:
            tip= i['Tipo']
            x.add(tip)
        numTipos = len(x)-1
        li2 = li2[1+2*numTipos + len(list_comis):]
    
    li2=empataLis(li1,li2)    # Comenzamos a procesar la ficha curricular
    
    apartado=""
    i=0
    while(i<len(li1)):
        if 'class="TitulosVerde"' in li1[i] and i+3<len(li1): # 'TitulosVerde' nos indica el comienzo de un
            if li2[i+1]!='.':#if li2[i+1]!='NA' and li2[i+1]!='.':              #  nuevo apartado de la ficha curricular
                apartado=li2[i]
                nom = apartado + ""
                exp[apartado]=[{nom:li2[i+1] + " - " + li2[i+2],"Fechas":li2[i+3]}] # Guardamos la experiencia con sus fechas
            i=i+4
        elif 'class="textoNegro"' in li1[i]:                  # 'textoNegro' nos indica que seguimos en el mismo apartado
            nom = apartado + ""
            exp[apartado].append({nom:li2[i] + " - " + li2[i+1],"Fechas":li2[i+2]}) # Guardamos la experiencia con sus fechas
            i=i+3
    return exp          

### Limpieza Votaciones

In [12]:
def limpiaVotacion(nombreVot):
    '''
        El propósito de esta función es generar un identificador único para cada una de las votaciones.
        En primer lugar, se limpia el texto eliminando algunas de las palabras más comunes. Posteriormente, 
        se eliminan signos de puntuación y se convierte todo el texto a minúsculas. Por último, se genera el identificador
        con la primera letra de cada palabra.
    '''
    nombreVot = nombreVot.replace('(', '')
    nombreVot = nombreVot.replace(')', '')
    nombreVot = nombreVot.replace(' SE ', ' ')
    nombreVot = nombreVot.replace('DECRETO', '')
    nombreVot = nombreVot.replace(' QUE ', ' ')
    nombreVot = nombreVot.replace(' SE ', ' ')
    nombreVot = nombreVot.replace(' DE ', ' ')
    nombreVot = nombreVot.replace(' A ', ' ')
    nombreVot = nombreVot.replace(' LA ', ' ')
    nombreVot = nombreVot.replace(' EL ', ' ')
    nombreVot = nombreVot.replace(' POR ', ' ')
    nombreVot = nombreVot.replace(' PARA ', ' ')
    nombreVot = nombreVot.replace(' LOS ', ' ')
    nombreVot = nombreVot.replace(' LAS ', ' ')
    nombreVot = nombreVot.replace(' Y ', ' ')
    nombreVot = nombreVot.replace(' E ', ' ')
    nombreVot = nombreVot.replace(' CON ', ' ')
    nombreVot = nombreVot.replace(' LEY ', ' ')
    nombreVot = nombreVot.replace(' EN ', ' ')
    nombreVot = nombreVot.replace(' LO ', ' ')
    nombreVot = nombreVot.replace(' DEL ', ' ')
    nombreVot = nombreVot.replace('Á', 'A')
    nombreVot = nombreVot.replace('É', 'E')
    nombreVot = nombreVot.replace('Í', 'I')
    nombreVot = nombreVot.replace('Ó', 'O')
    nombreVot = nombreVot.replace('Ú', 'U')
    nombreVot = nombreVot.replace('.', ' ')
    nombreVot = nombreVot.replace(';', ' ')
    nombreVot = nombreVot.replace(',', ' ')
    nombreVot = nombreVot.replace('  ', ' ')
    nombreVot = nombreVot.lower()
    nombreFin=""
    for s in nombreVot.split():
        nombreFin+=s[0]
    return nombreFin    

## Ejecución del Spider

Es aquí donde, después de declarar el Spider y las funciones de limpieza y procesamiento de datos, se echa a correr todo el proceso. Declaramos los 3 diccionarios que contendrán las bases de datos.

* *dipus* contendrá todos los perfiles básicos
* *por_votaciones* contará con todas las votaciones
* *diccionario_final* será el diccionario que contendrá los perfiles básicos y además las iniciativas, proposiciones, fichas curriculares y comisiones de todos los diputados. Este diccionario se crea después de la ejecución del spider a partir de la información contenida en *dic_proposiciones*, *dic_iniciativas*, *dic_experiencia_lista*, *dic_experiencia_texto* y *dic_comisiones*

In [13]:
dipus={}
por_votaciones = {}
diccionario_final={}
dic_proposiciones = {}
dic_iniciativas = {}
dic_experiencia_lista = {}
dic_experiencia_texto = {}
dic_comisiones = {}

urlDipu = "http://sitllxi.diputados.gob.mx/"  
urlCurr = "http://sitllxi.diputados.gob.mx/curricula.php?dipt="  #MODIFICAR ESTO AL PASAR A 60



terminaciones = []

for i in range(0,1):
    for j in range(1,2):
        for k in range(1,3):
            terminaciones.append(str(i)+str(j)+str(k))
terminaciones.append(str(500))
terminaciones.pop(0)




# Run the Spider
process = CrawlerProcess()
process.crawl(Diputados)
process.start()


2021-05-13 13:12:10 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-05-13 13:12:10 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Windows-10-10.0.19041-SP0
2021-05-13 13:12:10 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-05-13 13:12:10 [scrapy.crawler] INFO: Overridden settings:
{}
2021-05-13 13:12:10 [scrapy.extensions.telnet] INFO: Telnet Password: c79ae67225420774
2021-05-13 13:12:10 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-05-13 13:12:11 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddl

2021-05-13 13:12:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/proposiciones_diputados_xperiodonplxi.php?dipt=269> (referer: http://sitllxi.diputados.gob.mx/curricula.php?dipt=269)
2021-05-13 13:12:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/iniciativas_por_pernplxi.php?iddipt=264&pert=7> (referer: http://sitllxi.diputados.gob.mx/iniciativas_diputados_xperiodonplxi.php?dipt=264)
2021-05-13 13:12:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/iniciativas_por_pernplxi.php?iddipt=264&pert=1> (referer: http://sitllxi.diputados.gob.mx/iniciativas_diputados_xperiodonplxi.php?dipt=264)
2021-05-13 13:12:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/iniciativas_diputados_xperiodonplxi.php?dipt=269> (referer: http://sitllxi.diputados.gob.mx/curricula.php?dipt=269)
2021-05-13 13:12:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputado

2021-05-13 13:12:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/iniciativas_por_pernplxi.php?iddipt=269&pert=3> (referer: http://sitllxi.diputados.gob.mx/iniciativas_diputados_xperiodonplxi.php?dipt=269)
2021-05-13 13:12:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/votaciones_por_pernplxi.php?iddipt=266&pert=11> (referer: http://sitllxi.diputados.gob.mx/votaciones_diputados_xperiodonplxi.php?dipt=266)
2021-05-13 13:12:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/votaciones_por_pernplxi.php?iddipt=269&pert=5> (referer: http://sitllxi.diputados.gob.mx/votaciones_diputados_xperiodonplxi.php?dipt=269)
2021-05-13 13:12:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/votaciones_por_pernplxi.php?iddipt=266&pert=3> (referer: http://sitllxi.diputados.gob.mx/votaciones_diputados_xperiodonplxi.php?dipt=266)
2021-05-13 13:12:44 [scrapy.core.engine] DEBUG: C

2021-05-13 13:13:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/proposiciones_por_pernplxi.php?iddipt=266&pert=8> (referer: http://sitllxi.diputados.gob.mx/proposiciones_diputados_xperiodonplxi.php?dipt=266)
2021-05-13 13:13:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/proposiciones_por_pernplxi.php?iddipt=266&pert=7> (referer: http://sitllxi.diputados.gob.mx/proposiciones_diputados_xperiodonplxi.php?dipt=266)
2021-05-13 13:13:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/proposiciones_por_pernplxi.php?iddipt=266&pert=3> (referer: http://sitllxi.diputados.gob.mx/proposiciones_diputados_xperiodonplxi.php?dipt=266)
2021-05-13 13:13:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/proposiciones_por_pernplxi.php?iddipt=266&pert=1> (referer: http://sitllxi.diputados.gob.mx/proposiciones_diputados_xperiodonplxi.php?dipt=266)
2021-05-13 13:13:14 [scrapy.

2021-05-13 13:13:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/iniciativas_por_pernplxi.php?iddipt=263&pert=5> (referer: http://sitllxi.diputados.gob.mx/iniciativas_diputados_xperiodonplxi.php?dipt=263)
2021-05-13 13:13:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/iniciativas_por_pernplxi.php?iddipt=263&pert=3> (referer: http://sitllxi.diputados.gob.mx/iniciativas_diputados_xperiodonplxi.php?dipt=263)
2021-05-13 13:13:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/proposiciones_por_pernplxi.php?iddipt=261&pert=3> (referer: http://sitllxi.diputados.gob.mx/proposiciones_diputados_xperiodonplxi.php?dipt=261)
2021-05-13 13:13:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/votaciones_por_pernplxi.php?iddipt=261&pert=3> (referer: http://sitllxi.diputados.gob.mx/votaciones_diputados_xperiodonplxi.php?dipt=261)
2021-05-13 13:13:29 [scrapy.core.engine] D

2021-05-13 13:13:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/votaciones_por_pernplxi.php?iddipt=265&pert=5> (referer: http://sitllxi.diputados.gob.mx/votaciones_diputados_xperiodonplxi.php?dipt=265)
2021-05-13 13:13:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/votaciones_por_pernplxi.php?iddipt=265&pert=3> (referer: http://sitllxi.diputados.gob.mx/votaciones_diputados_xperiodonplxi.php?dipt=265)
2021-05-13 13:13:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/votaciones_por_pernplxi.php?iddipt=268&pert=9> (referer: http://sitllxi.diputados.gob.mx/votaciones_diputados_xperiodonplxi.php?dipt=268)
2021-05-13 13:13:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxi.diputados.gob.mx/votaciones_por_pernplxi.php?iddipt=268&pert=11> (referer: http://sitllxi.diputados.gob.mx/votaciones_diputados_xperiodonplxi.php?dipt=268)
2021-05-13 13:13:47 [scrapy.core.engine] DEBUG: Cra

### Creación de la base de datos
Primero procesamos toda la información para llenar *diccionario_final*. Los 3 diccionarios obtenidos se guardan en 3 archivos ***json*** por separado.

In [14]:
diccionario_final = {}
for key, value in dipus.items():
    legislador= obtieneExp(key,dic_experiencia_lista[key],dic_experiencia_texto[key],dic_comisiones[key], dic_iniciativas[key],dic_proposiciones[key],dipus.get(key,{}))
    diccionario_final[key] = legislador

In [15]:
diccionario_final

{'jesuseverardovillarrealsalinas': {'PERFIL GENERAL': [{'ClaveUnica': 'jesuseverardovillarrealsalinas',
    'Nombre': 'Jesús Everardo Villarreal Salinas',
    'TipoElección': 'Mayoría Relativa',
    'Entidad': 'Tamaulipas',
    'Partido': 'PRI',
    'Distrito': '2',
    'Curul': 'I-300',
    'Correo': 'jesus.villarreal@congreso.gob.mx ',
    'FechaNacimiento': '23-marzo-1978',
    'Suplente': 'Laura Felicitas García Dávila',
    'UltimoGrado': 'Mayoría Relativa ',
    'PaginaInternet': 'http://sitllxi.diputados.gob.mx/curricula.php?dipt=261'}],
  'INICIATIVAS': [{'NombreIniciativa': 'Proyecto de decreto que reforma el artículo 9°-A de la Ley de Coordinación Fiscal.',
    'TipoIniciativa': 'Suscribe',
    'FechaIniciativa': ' 15-Marzo-2011',
    'TurnoComisionI': 'NA',
    'SinopsisIniciativa': 'Actualizar el esquema de participación de fondos en beneficio de los municipios y entidades donde se ubiquen puentes, nacionales o internacionales, quienes podrán obtener hasta 50 por ciento del

In [ ]:
with open('BaseCurricula61.json', 'a') as f:
            json.dump(diccionario_final, f, indent=4, ensure_ascii=False)

In [ ]:
with open('PerfilesBasicos61.json', 'a') as f:
            json.dump(dipus, f, indent=4, ensure_ascii=False)

In [ ]:
with open('BaseVotaciones61.json', 'a') as f:
            json.dump(por_votaciones, f, indent=4, ensure_ascii=False)

De manera similar, si así se desea, los diccionarios se pueden convertir a lista de diccionarios y guardarlos en esa estructura.

In [ ]:
#list_perfiles_basicos = [value for value in dipus.values()]

In [ ]:
#list_of_votos = [value for value in por_votaciones.values()]

In [ ]:
#list_legisladores = [value for value in diccionario_final.values()]

In [ ]:
#with open('PerfilesBasicos_FormatoLista.json', 'a') as f:
#            json.dump(list_perfiles_basicos, f, indent=4, ensure_ascii=False)

In [ ]:
#with open('Curricula_FormatoLista.json', 'a') as f:
#            json.dump(list_legisladores, f, indent=4, ensure_ascii=False)

In [ ]:
#with open('Votaciones_FormatoLista.json', 'a') as f:
#            json.dump(list_of_votos, f, indent=4, ensure_ascii=False)

### Visualizando los datos
Esta sección sirve meramente como apoyo visual. Requiere que en la última sección los diccionarios *dipus*, *por_votaciones* y *diccionario_final* se pasen a estructura lista de diccionarios.

In [ ]:
# Visualizamos en formato Dataframe los perfiles básicos
perfil_legislador = pd.DataFrame(list_perfiles_basicos)
#perfil_legislador

In [ ]:
# Visualizamos en formato Dataframe las votaciones
perfil_votos = pd.DataFrame(list_of_votos)
#perfil_votos

In [ ]:
# En este corto apartado, se selecciona arbitrariamente un diputado en específico y se obtiene el nombre de cada una de
# las tablas de la cual se tiene información del diputado seleccionado

diputadoX=list_legisladores[2] # Seleccionamos un diputado

lista_tablas=[]
for key in diputadoX:
    print(key)
    df = pd.DataFrame(diputadoX[key])
    lista_tablas.append(df)

In [ ]:
# Con base en la selección del diputado, se puede consultar cada una de las tablas disponibles

df=lista_tablas[3] # Seleccionamos la tabla que deseamos visualizar

In [16]:
por_votaciones

{'ra37bcpafgp': {'ClaveUnicaVotacion': 'ra37bcpafgp',
  'NombreVotacion': 'DECRETO QUE REFORMA LOS ARTICULOS 37 Y 70 BIS DE LA LEY DE CAMINOS, PUENTES Y AUTOTRANSPORTE FEDERAL (EN LO GENERAL Y EN LO PARTICULAR) ',
  'FechaVotacion': '2 Febrero 2012',
  'baltazarmanuelhinojosaochoa': 'Ausente',
  'cristabellzamoracabrera': 'Ausente',
  'julianfranciscovelazquezyllorente': 'En contra',
  'javiergilortiz': 'Ausente',
  'edgardomelhemsalinas': 'Ausente',
  'josefranciscorabagocastillo': 'Ausente',
  'luisalejandroguevaracobos': 'Ausente',
  'oralialopezhernandez': 'En contra'},
 'cp1cmpauclcgegp': {'ClaveUnicaVotacion': 'cp1cmpauclcgegp',
  'NombreVotacion': 'DECRETO QUE CONCEDE PERMISO A 1 CIUDADANO MEXICANO PARA QUE PUEDA ACEPTAR Y USAR CONDECORACION QUE LE CONFIERE GOBIERNO EXTRANJERO (EN LO GENERAL Y EN LO PARTICULAR) ',
  'FechaVotacion': '2 Febrero 2012',
  'baltazarmanuelhinojosaochoa': 'Ausente',
  'cristabellzamoracabrera': 'Ausente',
  'julianfranciscovelazquezyllorente': 'A favo